In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
print(TAVILY_API_KEY[:4])

sk
tvly


#### 문제 4-1 : 카페 메뉴 도구(Tool) 호출 체인 구현 ( LangChain 사용)

In [4]:
import os
import re
from langchain_core.documents import Document
from langchain.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

cafe_menu_content = """
1. 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 뜨거운 물
   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.

2. 카페라떼
   • 가격: ₩5,500
   • 주요 원료: 에스프레소, 스팀 밀크
   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 질감과 부드러운 맛이 특징이며, 다양한 시럽과 토핑 추가가 가능합니다. 라떼 아트로 시각적 즐거움도 제공합니다.

3. 카푸치노
   • 가격: ₩5,000
   • 주요 원료: 에스프레소, 스팀 밀크, 우유 거품
   • 설명: 에스프레소, 스팀 밀크, 우유 거품이 1:1:1 비율로 구성된 이탈리아 전통 커피입니다. 진한 커피 맛과 부드러운 우유 거품의 조화가 일품이며, 계피 파우더를 뿌려 제공합니다.

4. 바닐라 라떼
   • 가격: ₩6,000
   • 주요 원료: 에스프레소, 스팀 밀크, 바닐라 시럽
   • 설명: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다.

5. 카라멜 마키아토
   • 가격: ₩6,500
   • 주요 원료: 에스프레소, 스팀 밀크, 카라멜 시럽, 휘핑크림
   • 설명: 스팀 밀크 위에 에스프레소를 부어 만든 후 카라멜 시럽과 휘핑크림으로 마무리한 달콤한 커피입니다. 카라멜의 진한 단맛과 커피의 깊은 맛이 조화를 이루며, 시각적으로도 아름다운 층을 형성합니다.

6. 콜드브루
   • 가격: ₩5,000
   • 주요 원료: 콜드브루 원액, 차가운 물
   • 설명: 찬물에 12-24시간 우려낸 콜드브루 원액을 사용한 시원한 커피입니다. 부드럽고 달콤한 맛이 특징이며, 산미가 적어 누구나 부담 없이 즐길 수 있습니다. 얼음과 함께 시원하게 제공됩니다.

7. 프라푸치노
   • 가격: ₩7,000
   • 주요 원료: 에스프레소, 우유, 얼음, 휘핑크림
   • 설명: 에스프레소와 우유, 얼음을 블렌더에 갈아 만든 시원한 음료입니다. 부드럽고 크리미한 질감이 특징이며, 휘핑크림을 올려 달콤함을 더했습니다. 여름철 인기 메뉴입니다.

8. 녹차 라떼
   • 가격: ₩5,800
   • 주요 원료: 말차 파우더, 스팀 밀크, 설탕
   • 설명: 고급 말차 파우더와 부드러운 스팀 밀크로 만든 건강한 음료입니다. 녹차의 은은한 쓴맛과 우유의 부드러움이 조화를 이루며, 항산화 성분이 풍부합니다. 달콤함 조절이 가능합니다.

9. 아이스 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 차가운 물, 얼음
   • 설명: 진한 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 아이스 커피입니다. 깔끔하고 시원한 맛이 특징이며, 원두 본연의 풍미를 느낄 수 있습니다. 더운 날씨에 인기가 높습니다.

10. 티라미수
    • 가격: ₩7,500
    • 주요 원료: 마스카포네 치즈, 에스프레소, 레이디핑거, 코코아 파우더
    • 설명: 이탈리아 전통 디저트로 마스카포네 치즈와 에스프레소에 적신 레이디핑거를 층층이 쌓아 만들었습니다. 부드럽고 달콤한 맛이 특징이며, 코코아 파우더로 마무리하여 깊은 풍미를 자랑합니다.
"""

with open("../data/cafe_menu.txt", "w", encoding="utf-8") as f:
    f.write(cafe_menu_content.strip())

loader = TextLoader("../data/cafe_menu.txt", encoding="utf-8")
documents = loader.load()

def split_menu_items(document: Document) -> list[Document]:
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    return menu_documents

# 문서 분할 실행
cafe_documents = []
for doc in documents:
    cafe_documents.extend(split_menu_items(doc))

# 임베딩 모델 및 FAISS 벡터 DB 생성
embeddings_model = OpenAIEmbeddings()
cafe_db = FAISS.from_documents(documents=cafe_documents, embedding=embeddings_model)

# DB 저장
db_path = "../db/cafe_db"
if not os.path.exists("../db"):
    os.makedirs("../db")
cafe_db.save_local(db_path)

print(f"총 {len(cafe_documents)}개의 메뉴 항목으로 벡터 DB를 생성하고 '{db_path}'에 저장했습니다.")


총 10개의 메뉴 항목으로 벡터 DB를 생성하고 '../db/cafe_db'에 저장했습니다.


In [8]:
import wikipedia
from langchain_core.tools import tool
from langchain_community.tools import TavilySearchResults
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 도구 1: Tavily 웹 검색 (변경 없음)
@tool
def tavily_search_func(query: str) -> str:
    """데이터베이스에 없거나 최신 정보가 필요할 때 웹에서 정보를 검색합니다."""
    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)
    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
    ])
    return formatted_docs if formatted_docs else "관련 정보를 찾을 수 없습니다."

# 도구 2: 위키피디아 검색 및 요약 체인 (수정된 부분)
def search_wiki_func(query: str) -> list[Document]:
    """wikipedia 라이브러리를 직접 사용하여 안정성을 높인 검색 함수"""
    wikipedia.set_lang("ko")
    try:
        # 가장 관련성 높은 페이지 제목 1개를 검색
        page_titles = wikipedia.search(query, results=1)
        if not page_titles:
            return [Document(page_content="위키피디아에서 관련 정보를 찾을 수 없습니다.")]
        
        # 해당 페이지의 요약 내용을 가져옴
        page = wikipedia.page(page_titles[0], auto_suggest=False)
        
        # LangChain Document 형식으로 반환
        return [Document(
            page_content=page.summary, 
            metadata={'source': page.url, 'title': page.title}
        )]
    except wikipedia.exceptions.PageError:
        return [Document(page_content=f"'{query}'에 대한 위키피디아 페이지를 찾을 수 없습니다.")]
    except wikipedia.exceptions.DisambiguationError as e:
        # 검색어가 모호할 경우 처리
        options = ", ".join(e.options[:3])
        return [Document(page_content=f"'{query}'는 여러 의미를 가집니다. 더 구체적인 검색어가 필요합니다. (예: {options})")]
    except Exception as e:
        return [Document(page_content=f"위키피디아 검색 중 오류 발생: {e}")]

summary_prompt = ChatPromptTemplate.from_template("다음 텍스트를 간결하게 요약해 주세요:\n\n{context}\n\n요약:")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

summary_chain = (
    # RunnableLambda에 search_wiki_func의 결과(리스트)를 문자열로 합치는 로직 추가
    {"context": RunnableLambda(search_wiki_func) | (lambda docs: "\n".join([doc.page_content for doc in docs]))}
    | summary_prompt
    | llm
    | StrOutputParser()
)

@tool
def wiki_summary(query: str) -> str:
    """커피 역사, 음료 제조법 등 일반적인 지식이나 배경 정보가 필요할 때 위키피디아에서 정보를 검색하고 요약합니다."""
    return summary_chain.invoke(query) # invoke 인자를 query 딕셔너리에서 문자열로 변경

# 도구 3: 카페 메뉴 DB 검색 (변경 없음)
@tool
def db_search_cafe_func(query: str) -> list[Document]:
    """암호화된 데이터베이스에서 레스토랑 메뉴 정보에 안전하게 접근하여 검색합니다. 데이터 기밀성을 유지하기 위해 메뉴 관련 쿼리에만 이 도구를 사용하세요."""
    docs = cafe_db.similarity_search(query, k=3)
    return docs if docs else [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

# 3개 도구를 리스트로 묶고 LLM에 바인딩
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]
llm_with_tools = llm.bind_tools(tools=tools)

print("3개의 도구가 성공적으로 정의되고 LLM에 바인딩되었습니다.")
for t in tools:
    print(f"- {t.name}: {t.description}")

3개의 도구가 성공적으로 정의되고 LLM에 바인딩되었습니다.
- tavily_search_func: 데이터베이스에 없거나 최신 정보가 필요할 때 웹에서 정보를 검색합니다.
- wiki_summary: 커피 역사, 음료 제조법 등 일반적인 지식이나 배경 정보가 필요할 때 위키피디아에서 정보를 검색하고 요약합니다.
- db_search_cafe_func: 암호화된 데이터베이스에서 레스토랑 메뉴 정보에 안전하게 접근하여 검색합니다. 데이터 기밀성을 유지하기 위해 메뉴 관련 쿼리에만 이 도구를 사용하세요.


In [9]:
from langchain_core.runnables import RunnableConfig, chain
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from pprint import pprint

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 카페 메뉴와 음식 관련 지식을 제공하는 AI 어시스턴트입니다."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def cafe_chain(user_input: str, config: RunnableConfig):
    """사용자 입력을 받아 도구를 호출하고 최종 답변을 생성하는 체인"""
    input_ = {"user_input": user_input}
    
    # 1. LLM을 호출하여 어떤 도구를 사용할지 결정
    ai_msg = llm_chain.invoke(input_, config=config)
    
    tool_msgs = []
    # 2. 결정된 도구 실행
    for tool_call in ai_msg.tool_calls:
        print(f"Calling Tool: {tool_call['name']} with args: {tool_call['args']}")
        if tool_call["name"] == "tavily_search_func":
            tool_output = tavily_search_func.invoke(tool_call['args'], config=config)
        elif tool_call["name"] == "wiki_summary":
            tool_output = wiki_summary.invoke(tool_call['args'], config=config)
        elif tool_call["name"] == "db_search_cafe_func":
            tool_output = db_search_cafe_func.invoke(tool_call['args'], config=config)
        else:
            tool_output = "알 수 없는 도구입니다."
            
        tool_msgs.append(ToolMessage(content=str(tool_output), tool_call_id=tool_call['id']))
    
    # 3. 도구 실행 결과를 포함하여 최종 답변 생성
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

print("카페 메뉴 정보 제공 체인이 준비되었습니다.")

카페 메뉴 정보 제공 체인이 준비되었습니다.


In [11]:
# 테스트 질문
test_query = "아메리카노의 가격과 특징은 무엇인가요?"

# 체인 실행
response = cafe_chain.invoke(test_query)

# 결과 출력
print("\n--- 최종 답변 ---")
pprint(response.content)

Calling Tool: db_search_cafe_func with args: {'query': '아메리카노'}
Calling Tool: wiki_summary with args: {'query': 'Americano'}

--- 최종 답변 ---
('아메리카노는 에스프레소에 뜨거운 물을 추가하여 만든 커피 음료입니다. 이 음료는 에스프레소의 진한 맛을 부드럽게 하여 마시기 쉽게 만들어 '
 '줍니다. 아메리카노는 일반적으로 "caffè americano"라는 이름으로도 알려져 있으며, "American coffee"라는 의미를 '
 '가지고 있습니다.\n'
 '\n'
 '### 가격\n'
 '아메리카노의 가격은 약 4,500원입니다.\n'
 '\n'
 '### 특징\n'
 '- **제조 방법**: 에스프레소에 뜨거운 물을 추가하여 만듭니다.\n'
 '- **맛**: 에스프레소의 진한 맛을 부드럽게 하여 마시기 쉽게 만들어 줍니다.\n'
 '- **유래**: 아메리카노는 제2차 세계대전 중 미국 군인들이 이탈리아에서 에스프레소를 마시기 어려워 물을 추가하여 만든 음료에서 '
 '유래되었습니다.\n'
 '\n'
 '아메리카노는 커피의 깊은 맛을 즐기고 싶지만, 에스프레소의 강한 맛이 부담스러운 분들에게 적합한 음료입니다.')
